## <center>Error for </center>

\begin{cases}
y'(t)= ay(t-\tau),&\\
y(t)=1 for all t<0&
\end{cases}

## <center>with EE </center>

(Take a=\pi / 2)

In [1]:
import numpy as np

## Linear interpolation for delay terms
def y_delayed(t, y, t_q):
    j = np.searchsorted(t, t_q)  #Busca el índice donde insertar t_q para mantener el arreglo ordenado.
    j1, j2 = j-1, j     #Toma los dos puntos de la malla que rodean a t_q
    w = (t_q - t[j1]) / (t[j2] - t[j1])      #Aplica interpolación en estas 2 líneas
    y_delayed = (1 - w) * y[j1] + w * y[j2]
    return y_delayed

## Explicit Euler
def EE(f, phi, t_0, t_f, tau, N, history=False):

    ## Preliminaries
    h = (t_f - t_0) / N
    t = np.linspace(t_0, t_f, N+1)
    y = np.zeros(len(t))
    y[0] = phi(t_0)

    ## EE with linear interpolation for delay term
    for i in range(0, N):   #AQUI ESTÁ LA PARTE DEL STEP METHOD
        if t[i] < tau:
            y[i+1] = y[i] + h*f(t[i], y[i], phi(t[i] - tau)) ## We have exact values for these delay terms, let's use them
        else:
            y[i+1] = y[i] + h*f(t[i], y[i], y_delayed(t, y, t[i] - tau)) ## We need to use linear interpolation for these delay terms

    if history == True: ## Wether the solution should include the history (values on the interval [t_0 - tau, t_0])
        t_hist = np.linspace(t_0-tau, t_0, int(N / (t_f // tau) + 1))
        y_hist = np.array([phi(ti) for ti in t_hist])
        t = np.concatenate((t_hist, t))
        y = np.concatenate((y_hist, y))

    return h, t, y

In [2]:
from math import factorial

## Right-hand-side of the simple DDE
def simple(a):
    def simple_dde(t, y1, y2):
        return -a*y2
    return simple_dde


## Constant one history function
def phi_0(t):
    return 1

## Exact solution FOR THE SIMPLE DDE ONLY with constant 1 history function, starting at t_0 = 0:
## Note that this doesn't work for arbitrarily large time intervals, because the terms in the sum blow up,
##    leading to errors caused by floating-point arithmetic.
def simple_exact(a, t_0, t_f, tau, N): #The function computes the EXACT solution of the delay differential equation
    #ESTO ES PARA EL CASO CONCRETO DE ECUACIÓN SIMPLE, OSEA y′(t)=ay(t−τ),y(t)=1 for t≤0
    #PARA OTROS SISTEMAS DE ECUACIONES SE DEFINE DIFERENTE!!!
    if t_0 != 0:
        raise ValueError("t_0 should be set to 0")
    t = np.linspace(t_0, t_f, N+1)
    y = np.zeros(len(t))
    for i, ti in enumerate(t):
        n = int(np.floor(ti / tau)) + 1 #Because the interval index is determined by how many delays fit into t 
        #Osea lo que hacemos es dividir el intervalo, 0<t<tau, tau<t<2tau,.... (estamos evaluando y(t-tau))
        # El primer subintervalo ([-tau,0]) lo calculamos directamente 
        #cada siguiente subintervalo lo calculamos con el anterior
        y[i] = 1
        for k in range(1, n+1):
            y[i] += (-a)**k * (ti - (k-1)*tau)**k / factorial(k)
    return y

In [4]:
#Initial values
a = np.pi / 2
t_0 = 0
t_f = 30
tau = 1


maxnormresult = []
orderresult = []

N = 45 #I changed N


    
for i in range(5,10):
    h, t, y = EE(simple(a), phi_0, t_0, t_f, tau, N*2**i, False)
    exact = simple_exact(a, t_0, t_f, tau, N*2**i)
    maxnorm = np.linalg.norm(exact - y,np.inf)
    maxnormresult.append(maxnorm)
    
errorvector = np.hstack(maxnormresult)  #EL ERROR VECTOR ES UN VECTOR DE ERROR QUE MUESTRA EL ERROR ENTRE LA SOLUCIÓN EXACTA
#Y LA SOLUCIÓN PARA N=45*2**0,45*2**1,...

## R(h) and p as log_2(R(h))
for i in range(0,len(errorvector)-1):
    order = np.log2(errorvector[i]/errorvector[i+1])
    orderresult.append(order)
    
print(orderresult)
print(errorvector)

[np.float64(1.06936107880301), np.float64(1.035495931568563), np.float64(1.017950347518892), np.float64(1.0090233771502763)]
[0.4575261  0.21802491 0.10636305 0.05252393 0.02609822]


Order of convergence is 1 as in ODEs

Error tends to cero as N increases

However if N is not big the aproximation is not good

